# Analyze Speech & Language with Google API

## Task 1. Create an API key

Generate an API user key to pass in the request URL.

1. To create an API key, select **Navigation menu > APIs & Services > Credentials**.

2. Click **Create credentials** at the top and select **API key**:

3. Copy the API key to a text file to use in a later step. Click **Close**.

## Task 2. Make an entity analysis request and call the Natural Language API

1. Click **Navigation menu > Compute Engine** to connect via SSH to the VM instance.

2. Click on the **SSH** button.

3. In the command line, enter in the following:
   export API_KEY=<YOUR_API_KEY?>

4. Create a JSON file using the code that follows.

In [ ]:
cat > nl_request.json <<EOF
{
  "document":{
    "type":"PLAIN_TEXT",
    "content":"With approximately 8.2 million people residing in Boston, the capital city of Massachusetts is one of the largest in the United States."
  },
  "encodingType":"UTF8"
}
EOF

5. Pass the following request to the **Natural Language API** using the **curl** command, and save the response in a file.

In [ ]:
curl "https://language.googleapis.com/v1/documents:analyzeEntities?key=${API_KEY}" \
  -s -X POST -H "Content-Type: application/json" --data-binary @nl_request.json > nl_response.json

cat nl_response.json

## Task 3. Create a speech analysis request and call the Speech API

1. Create a JSON file using the code that follows.

In [ ]:
cat > speech_request.json <<EOF
{
  "config": {
      "encoding":"FLAC",
      "languageCode": "en-US"
  },
  "audio": {
      "uri":"gs://cloud-samples-tests/speech/brooklyn.flac"
  }
}
EOF

2. Pass the following request to the **Natural Language API** using the **curl** command, and save the response in a file.

In [ ]:
curl -s -X POST -H "Content-Type: application/json" --data-binary @speech_request.json \
"https://speech.googleapis.com/v1/speech:recognize?key=${API_KEY}" > speech_response.json

cat speech_response.json

## Task 4. Analyze sentiment with the Natural Language API

1. Create a py file using the code that follows.

In [ ]:
cat > sentiment_analysis.py <<EOF
import argparse

from google.cloud import language_v1

def print_result(annotations):
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude

    for index, sentence in enumerate(annotations.sentences):
        sentence_sentiment = sentence.sentiment.score
        print(
            "Sentence {} has a sentiment score of {}".format(index, sentence_sentiment)
        )

    print(
        "Overall Sentiment: score of {} with magnitude of {}".format(score, magnitude)
    )
    return 0

def analyze(movie_review_filename):
    """Run a sentiment analysis request on text within a passed filename."""
    # Instantiate a LanguageServiceClient instance as the client.
    client = language_v1.LanguageServiceClient()

    # Read the filename containing the text data into a variable.
    with open(movie_review_filename) as file:
        content = file.read()

    # Instantiate a Document object with the contents of the file.
    document = language_v1.Document(content=content,
        type_=language_v1.Document.Type.PLAIN_TEXT)

    # Call the client's analyze_sentiment method.
    annotations = client.analyze_sentiment(request={"document": document})

    # Print the results
    print_result(annotations)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description=__doc__, formatter_class=argparse.RawDescriptionHelpFormatter
    )
    parser.add_argument(
        "movie_review_filename",
        help="The filename of the movie review you'd like to analyze.",
    )
    args = parser.parse_args()

    analyze(args.movie_review_filename)
EOF

2. Run the sentiment analysis on the files.

In [ ]:
gsutil cp gs://cloud-samples-tests/natural-language/sentiment-samples.tgz .

gunzip sentiment-samples.tgz
tar -xvf sentiment-samples.tar
python3 sentiment_analysis.py reviews/bladerunner-pos.txt